In this notebook we import 'twinlab' for the first time and create a basic model.

First we import the necessary libraries: `numpy` allows us to efficiently manipulate arrays of numerical data; `pandas` gives us access to `DataFrames` which are a way of storing tabular data in `Python` and is the format used by `twinLab`. `matplotlib.pyplot` is used for plotting. `twinlab` is the machine-learning library we are using. Some of the libraries are renamed using `as` for convenience. 

In [ ]:
# Standard imports
import os

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Project imports
import twinlab as tl

The `twinLab-Demos` repository contains a number of sample datasets; in this cell we define the file paths for a basic dataset which will be used for this example. 

In [ ]:
# File paths
campaign_dir = os.path.join("resources", "campaigns", "basic")
datasets_dir = os.path.join("resources", "datasets")
file_train = os.path.join(datasets_dir, "basic.csv")
file_eval = os.path.join(campaign_dir, "eval.csv")
file_params = os.path.join(campaign_dir, "params.json")

# Campaign id
dataset_id = "basic"
campaign_id = "basic"

By using `tl.upload_dataset` we upload the dataset to the cloud so it can be used to train a model.

In [ ]:
# Upload dataset to the cloud
tl.upload_dataset(file_train, dataset_id, verbose=True)

To confirm that our dataset was succefully upload we can use `tl.list_datasets`.

In [ ]:
# List datasets
_ = tl.list_datasets(verbose=True)

We then train our surrogate model, `tl.train_campaign` can be used to train any model type availabel in `twinLab`, the type can be determined using the `campaign_id`. The keyword `file_params` is used to define the dataset that the model is trained on.

In [ ]:
# Start a new campaign and train a surrogate model
tl.train_campaign(file_params, campaign_id, verbose=True)

We can then use `tl.list_campaigns` to check that our model has been trained and exists in the cloud.

In [ ]:
# List campaigns
_ = tl.list_campaigns(verbose=True)

The `tl.query_campaign` function can be used to view all the parameters of a model.

In [ ]:
# Look at the status of a campaign
_ = tl.query_campaign(campaign_id, verbose=True)

By calling `tl.predict_campaign` with our `campaign_id` we can use our trained model to calculate a prediction and corresponding standard deviation. These results are saved to a csv file with a path defined by `file_eval`.

In [ ]:
# Predict using the trained emulator
df_mean, df_std = tl.predict_campaign(file_eval, campaign_id, verbose=True)

In [ ]:
# Load training and evaluation data (for plotting)
df_train = pd.read_csv(file_train)
df_eval = pd.read_csv(file_eval)

In [ ]:
# Plot parameters
nsigs = [1, 2]
# nsigs = [0.674, 1.960, 2.576]
color = "blue"
alpha = 0.5
plot_training_data = True
plot_model_mean = True
plot_model_bands = True

# Plot results
grid = df_eval["x"]
mean = df_mean["y"]
err = df_std["y"]
if plot_model_bands:
    label = r"Model prediction"
    plt.fill_between(grid, np.nan, np.nan, lw=0, color=color, alpha=alpha, label=label)
    for isig, nsig in enumerate(nsigs):
        plt.fill_between(grid, mean-nsig*err, mean+nsig*err, lw=0, color=color, alpha=alpha/(isig+1))
if plot_model_mean:
    label = r"Model prediction" if not plot_model_bands else None
    plt.plot(grid, mean, color=color, alpha=alpha, label=label)
if plot_training_data:
    plt.plot(df_train["x"], df_train["y"], ".", color="black", label="Training data")
plt.xlim((0.0, 1.0))
plt.xlabel(r"$X$")
plt.ylabel(r"$y$")
plt.legend()
plt.show()

When we are finished with our twinLab campaign we can delete the model and dataset from the cloud.

In [ ]:
# Delete campaign and dataset (if desired)
tl.delete_campaign(campaign_id, verbose=True)
tl.delete_dataset(dataset_id, verbose=True)